# GridPath Workshop (UCSB)

Initialization

In [ ]:
import os
import sys
import pandas as pd

# Print the current working directory. The result should end with state_model
software_dir = os.getcwd()
os.chdir("..")
print(software_dir)

In [ ]:
#Setting this as the primary directory for all future use
# This is be the main folder where all files are created and stored in. 
directory = os.getcwd()
print(directory)

In [ ]:
# Defining the database directory - where the database file will be created and stored. 
db_dir = f"{directory}/db"

# Defining CSV inputs directory - where the state level input files are stored. 
csv_dir = f"{directory}/input_csvs_gridpath"

#Defining scenario folder location
scen_dir = f"{directory}/scenarios"

# GridPath: Capacity Expansion
The following section is for running a production cost model in GridPath.

## User edit section
Add the folder name with input csvs and provide a name for this database.

In [ ]:
# Speciific  model for this run.
# Add the name of the specific mode folder file by replacing the worth between " " 
input_csvs_gridpath = "TN_7-2050-12_simple"

# Add your database name below between "" or leave blank to use input_csvs_gridpath as database name
db_name = ""
db_name = db_name if db_name else input_csvs_gridpath
print(f"Database name is set to: {db_name}")

In [ ]:
!python {directory}/gridpath/db/create_database.py --database {db_dir}/{db_name}.db
!python {directory}/gridpath/db/utilities/port_csvs_to_db.py --database {db_dir}/{db_name}.db --csv_location {csv_dir}/{input_csvs_gridpath}
!python {directory}/gridpath/db/utilities/scenario.py --database {db_dir}/{db_name}.db --csv_path {csv_dir}/{input_csvs_gridpath}/scenarios.csv

In [ ]:
# This will print out all possible scenarios available
scenario_csv = pd.read_csv(f"{csv_dir}/{input_csvs_gridpath}/scenarios.csv")
column_names = scenario_csv.columns[1:]

# Print scenario names
print("Available scenarios:")
for name in column_names:
    print(name)

In [ ]:
# Add or remove scenarios from above to the list below:
scen_list = ["VRElow_STlow_CONVmid_BPHS_NuclearPVboth_45Ret_PRM110_TN_3_2030_12_simple",
             "VREmid_STmid_CONVmid_BPHS_NuclearPVboth_45Ret_PRM110_TN_3_2030_12_simple",
             "VREhigh_SThigh_CONVmid_BPHS_NuclearPVboth_45Ret_PRM110_TN_3_2030_12_simple"]

In [ ]:
for selected_scenario in scen_list:
    !python {directory}/gridpath/gridpath/run_end_to_end.py --database {db_dir}/{db_name}.db --scenario {selected_scenario} --scenario_location {scen_dir}

# Visualization: Capacity Expansion
The following section is for visualizing the results from the GridPath Capacity Expansion model above.

In [ ]:
# Checking the directory is correct - should be state_model folder
os.chdir(software_dir)
print(os.getcwd())

In [ ]:
# loading packages and setting results folder
from loading_utils import * 
from plotting_utils import * 
from processing_utils import * 

path_to_scenarios = f"{directory}/scenarios"
print(path_to_scenarios)

In [ ]:
scen_labels_     = pd.read_csv(directory + '/input_csvs_plotting/scenario_labels_capex.csv') 
tech_labels_     = pd.read_csv(directory + '/input_csvs_plotting/technology_labels_capex.csv') 
dispatch_labels_ = pd.read_csv(directory + '/input_csvs_plotting/dispatch_labels_capex.csv') 

In [ ]:
capacity_         = _load_capacity(scen_labels_, path_to_scenarios, gp_model = 'capex')
grouped_capacity_ = _group_capacity_technologies(capacity_, tech_labels_)
grouped_capacity_

In [ ]:
_plot_new_and_existing_capacity(grouped_capacity_, scen_labels_, tech_labels_,
                                units        = 1e3,
                                units_label  = 'Existing & New Capacity (GW)',
                                legend       = True,
                                save         = True,
                                y_grid_inc   = 25,
                                title        = '',
                                file_name    = directory + r'/plots/group_installed_capacity_capex.pdf')

In [ ]:
dispatch_         = _load_energy_dispatch(scen_labels_, path_to_scenarios, gp_model = 'capex')
grouped_dispatch_ = _group_dispatch_technologies(dispatch_, tech_labels_)
grouped_dispatch_

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_dispatch(grouped_dispatch_, scen_labels_,  tech_labels_, 
               units        = 1e6, 
               units_label  = r'Electricity Generation (TWh)',
               save         = True,
               legend       = True,
               y_grid_inc   = 50,
               title        = '',
               file_name    = directory + r'/plots/group_electricity_generation_capex.pdf')

In [ ]:
ed_ = _processing_energy_dispatch(scen_labels_, path_to_scenarios, model = 'capex')
ed_ = _group_dispatch_technologies_by_zone_and_date_production(ed_, tech_labels_)

In [ ]:
# Plot energy dispatch for a given day
_plot_zone_energy_dispatch_production(ed_, scen_labels_, tech_labels_, dispatch_labels_,
                                      save      = True,
                                      legend    = True,
                                      file_name = directory + r'/plots/energy_dispatch_capex.pdf')

In [ ]:
emissions_, demand_ = _load_GHG_emissions(scen_labels_, path_to_scenarios, gp_model = 'capex')
emissions_

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_emissions(emissions_, scen_labels_, 
                units      = 1e6,
                unit_label = r'GHG Emissions (MtCO$_2$)',
                save       = True,
                legend     = True,
                title      = '',
                file_name  = directory + r'/plots/GHG_emissions_capex.pdf')

In [ ]:
intesity_, demand_ = _GHG_emissions_intensity(emissions_, demand_)
intesity_

Plot system Levelized Cosst of Electricity (LCOE) for different scenarios

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_emissions_intensity(intesity_, scen_labels_, 
                          unit_label = r'GHG Intensity (MtCO$_2$/MWh)',
                          save       = True,
                          legend     = True,
                          title      = '',
                          file_name  = directory + r'/plots/GHG_emissions_intensity_capex.pdf')

In [ ]:
system_cost_ = _load_system_cost(scen_labels_, path_to_scenarios, gp_model = 'capex')
system_cost_

In [ ]:
# Enter a title of your choice between the quotation marks ('') under title = ''.
_plot_system_cost(system_cost_, scen_labels_, 
                  unit_label = r'Costs (USD per MWh)', 
                  save       = True, 
                  legend     = True,
                  title      = '', 
                  file_name  = directory + r'/plots/system_cost_capex.pdf')

In [ ]:
# Replace with a specific scenario if needed, otherwise leave blank
scenario = 'VRElow_STlow_CONVmid_BPHS_NuclearPVboth_45Ret_PRM110_TN_3_2030_12_simple' 
# Replace with 'new' or existing' so only new capacity is shown, otherwise leave blank for new and existing
status = 'new' 
filtered_capacity_ = _filter_capacity(grouped_capacity_, scenario, status)
filtered_capacity_